In [10]:
import markovify, snscrape, json, os
import pandas as pd

In [74]:
os.system("snscrape --jsonl --max-results 100000 twitter-search 'from:OccupyDemocrats'> user-tweets.json")

0

In [80]:
tweets_df = pd.read_json('user-tweets.json', lines=True)

In [81]:
tweets_df.shape

(27340, 28)

In [76]:
tweets_df.iloc[0:2,:]

,_type,url,date,content,renderedContent,id,user,replyCount,retweetCount,likeCount,...,media,retweetedTweet,quotedTweet,inReplyToTweetId,inReplyToUser,mentionedUsers,coordinates,place,hashtags,cashtags
0,snscrape.modules.twitter.Tweet,https://twitter.com/OccupyDemocrats/status/157...,2022-10-09 20:55:32+00:00,@pmbakid I sent this to them and they said tha...,@pmbakid I sent this to them and they said tha...,1579213999752683521,"{'_type': 'snscrape.modules.twitter.User', 'us...",1,0,0,...,None,NaN,None,1.579212e+18,"{'_type': 'snscrape.modules.twitter.User', 'us...","[{'_type': 'snscrape.modules.twitter.User', 'u...",None,None,None,NaN
1,snscrape.modules.twitter.Tweet,https://twitter.com/OccupyDemocrats/status/157...,2022-10-09 20:42:34+00:00,BREAKING: Future Twitter owner Elon Musk prais...,BREAKING: Future Twitter owner Elon Musk prais...,1579210737226252288,"{'_type': 'snscrape.modules.twitter.User', 'us...",27,311,479,...,None,NaN,None,NaN,None,None,None,None,None,NaN


In [82]:
tweets_df = tweets_df[(tweets_df.inReplyToUser.isnull())&(tweets_df.content.str.contains('RT IF'))].reset_index(drop=True)
tweets_df.shape

(3818, 28)

In [83]:
tweets = pd.DataFrame([(a, b) for a, b in tweets_df.content.str.split('RT IF')], columns=['content1','content2'])
tweets

,content1,content2
0,BREAKING: Future Twitter owner Elon Musk prais...,MUSK AND KANYE SUCK!
1,BREAKING: Twitter competitor https://t.co/IO2p...,YOU SUPPORT TRIBEL!
2,BREAKING: Right-wing billionaire Elon Musk whi...,YOU DON’T BLAME HER!
3,BREAKING NEWS: Top Democrats introduce a bill ...,YOU SUPPORT THIS BILL!
4,BREAKING: MAGA panics as the woman whose abort...,YOU THINK THAT WALKER’S “PRO-LIFE” BELIEFS AR...
...,...,...
3813,BREAKING NEWS: Marjorie Taylor Greene tells he...,@mtgreenee MUST BE EXPELLED!
3814,BREAKING: Matt Gaetz's future sister-in-law ou...,YOU AGREE WITH HER!
3815,BREAKING: After having two venues cancel their...,YOU’RE PROUD OF CALIFORNIA FOR STANDING UP TO...
3816,BREAKING: @mattgaetz and @mtgreenee’s Saturday...,EVERY CITY IN THE REST OF THEIR TOUR SHOULD D...


In [84]:
text_model1 = markovify.NewlineText(tweets.content1, 3)
text_model2 = markovify.NewlineText(tweets.content2, 1)
for i in range(10):
    print((text_model1.make_short_sentence(200)+'RT IF'+text_model2.make_short_sentence(40)))

BREAKING: New campaign disclosures reveal that Donald Trump could face five years in federal prison. RT IF YOU SUPPORT THIS IS A BIGOT!
BREAKING: Two advisors to Donald Trump by shooting down his desperate attempt to help Trump steal the election, in criminal content for defying its subpoena. RT IF YOU THINK THAT TIM RYAN CRUSHES RUBIO!
BREAKING NEWS: Dr. Fauci announces his support of vaccine mandates for employees &amp; school mask mandates dead in its tracks — a major victory for the administration. RT IF YOU’RE WITH A WAR CRIMINAL!
BREAKING: The Department of Justice impanels a grand jury indicts him for refusing to get vaccinated — a no-option mandate that will affect about 148,000 workers. RT IF YOU FULLY SUPPORT THIS!
BREAKING: The Supreme Court slaps down a request from the Biden White House is set to announce that 95% of the federal Logan Act. RT IF YOU THINK RUSSIA SHOULD BE REJECTED!
BREAKING: MAGA candidate Dr. Oz under the bus, cancels over $5 million a year. RT IF YOU THI

In [101]:
n           = 50000
tweets_1    = [text_model1.make_short_sentence(200) for i in range(n)]
tweets_1    = [t for t in tweets_1 if t is not None]
tweets_2    = [text_model2.make_short_sentence(40)  for i in range(n)]
tweets_2    = [t for t in tweets_2 if t is not None]

n_actual    = min((len(tweets_1), len(tweets_2)))
tweets      = [tweets_1[i]+'RT IF'+tweets_2[i] for i in range(n_actual)]
tweets.append('BREAKING NEWS: President Biden announces that he has had a second child with his own face carved into it because he thinks climate change is a hoax. RT IF YOU THINK THE BIDEN TO VOTE FOR GOOD!')
tweets_json = {"tweets": [{"tweet":t} for t in tweets]}

In [102]:
with open('occupy_tweets.json','w') as outfile:
    json.dump(tweets_json, outfile)

BREAKING NEWS: President Biden announces that he has had a second child with his own face carved into it because he thinks climate change is a hoax. RT IF YOU THINK THE BIDEN TO VOTE FOR GOOD!